In [1]:
# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [7]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

/Users/bhawesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/bhawesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/bhawesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/bhawesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/frame

In [10]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [11]:
# Import Keras
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from tensorflow.keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [12]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

Using TensorFlow backend.


In [13]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 64

In [14]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [16]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/Users/bhawesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
/Users/bhawesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [17]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [18]:
X_train.shape

(7352, 128, 9)

- Defining the Architecture of LSTM

In [19]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(32,return_sequences=True,input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))

model.add(LSTM(28,input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 28)                6832      
_________________________________________________________________
dropout_2 (Dropout)          (None, 28)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 174       
Total params: 12,382
Trainable params: 12,382
Non-trainable params: 0
_________________________________________________________________


In [73]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [74]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 258s 35ms/step - loss: 1.3024 - accuracy: 0.4498 - val_loss: 0.9632 - val_accuracy: 0.6220
Epoch 2/30
7352/7352 [==============================] - 236s 32ms/step - loss: 0.8855 - accuracy: 0.5652 - val_loss: 0.8105 - val_accuracy: 0.6502
Epoch 3/30
7352/7352 [==============================] - 235s 32ms/step - loss: 0.7803 - accuracy: 0.6270 - val_loss: 0.7567 - val_accuracy: 0.6423
Epoch 4/30
7352/7352 [==============================] - 235s 32ms/step - loss: 0.7245 - accuracy: 0.6662 - val_loss: 0.6691 - val_accuracy: 0.7129
Epoch 5/30
7352/7352 [==============================] - 1078s 147ms/step - loss: 0.6872 - accuracy: 0.6834 - val_loss: 0.6223 - val_accuracy: 0.7258
Epoch 6/30
7352/7352 [==============================] - 265s 36ms/step - loss: 0.6280 - accuracy: 0.7338 - val_loss: 0.6051 - val_accuracy: 0.7489
Epoch 7/30
7352/7352 [==============================] - 261s 36ms/st

In [75]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  4      420        46        0                   0   
STANDING                 0      132       400        0                   0   
WALKING                  0        0         0      469                  21   
WALKING_DOWNSTAIRS       0        0         0       12                 401   
WALKING_UPSTAIRS         0        2         1       13                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                           21  
STANDING                           0  
WALKING                            6  
WALKING_DOWNSTAIRS                 7  
WALKING_UPSTAIRS                 455  


In [76]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 18s 6ms/step


In [77]:
score

[0.4158135156924696, 0.9100780487060547]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

### Assignment

### Changing LSTM

In [116]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [118]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.4))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [119]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [120]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 168s 23ms/step - loss: 1.2421 - accuracy: 0.4939 - val_loss: 1.0321 - val_accuracy: 0.6074
Epoch 2/30
7352/7352 [==============================] - 196s 27ms/step - loss: 0.8385 - accuracy: 0.6436 - val_loss: 0.8230 - val_accuracy: 0.6308
Epoch 3/30
7352/7352 [==============================] - 175s 24ms/step - loss: 0.6884 - accuracy: 0.6912 - val_loss: 0.6510 - val_accuracy: 0.7435
Epoch 4/30
7352/7352 [==============================] - 178s 24ms/step - loss: 0.5859 - accuracy: 0.7564 - val_loss: 0.5855 - val_accuracy: 0.7611
Epoch 5/30
7352/7352 [==============================] - 149s 20ms/step - loss: 0.4952 - accuracy: 0.7969 - val_loss: 0.5028 - val_accuracy: 0.7967
Epoch 6/30
7352/7352 [==============================] - 145s 20ms/step - loss: 0.4476 - accuracy: 0.8411 - val_loss: 0.5677 - val_accuracy: 0.7869
Epoch 7/30
7352/7352 [==============================] - 144s 20ms/step

In [121]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 513        0        24        0                   0   
SITTING                  0      425        64        0                   1   
STANDING                 0      117       405        2                   0   
WALKING                  0        0         1      386                  49   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        4         1        4                  31   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           8  
WALKING                           60  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 431  


In [122]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 11s 4ms/step


In [123]:
score

[0.751543751711401, 0.8754665851593018]

In [124]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 64

In [125]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.55))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 64)                18944     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [126]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [127]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 335s 46ms/step - loss: 1.2323 - accuracy: 0.4727 - val_loss: 1.0838 - val_accuracy: 0.5517
Epoch 2/30
7352/7352 [==============================] - 253s 34ms/step - loss: 0.8582 - accuracy: 0.6133 - val_loss: 0.7665 - val_accuracy: 0.6634
Epoch 3/30
7352/7352 [==============================] - 210s 29ms/step - loss: 0.7849 - accuracy: 0.6678 - val_loss: 0.7073 - val_accuracy: 0.7150
Epoch 4/30
7352/7352 [==============================] - 237s 32ms/step - loss: 0.6339 - accuracy: 0.7333 - val_loss: 0.6145 - val_accuracy: 0.7737
Epoch 5/30
7352/7352 [==============================] - 270s 37ms/step - loss: 0.5641 - accuracy: 0.7807 - val_loss: 0.4794 - val_accuracy: 0.8276
Epoch 6/30
7352/7352 [==============================] - 275s 37ms/step - loss: 0.4266 - accuracy: 0.8621 - val_loss: 0.3470 - val_accuracy: 0.8782
Epoch 7/30
7352/7352 [==============================] - 211s 29ms/step

In [131]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 535        0         0        0                   0   
SITTING                  0      416        67        0                   0   
STANDING                 0       96       434        1                   0   
WALKING                  0        0         0      491                   3   
WALKING_DOWNSTAIRS       0        0         0        6                 413   
WALKING_UPSTAIRS         0        0         1       15                   8   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             2  
SITTING                            8  
STANDING                           1  
WALKING                            2  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 447  


In [132]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 15s 5ms/step


In [133]:
score

[0.27563241636243435, 0.92840176820755]

### Changing Dropout

In [41]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [42]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.7))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [44]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 131s 18ms/step - loss: 1.3737 - accuracy: 0.4276 - val_loss: 1.1522 - val_accuracy: 0.4951
Epoch 2/30
7352/7352 [==============================] - 118s 16ms/step - loss: 1.0582 - accuracy: 0.5358 - val_loss: 0.9802 - val_accuracy: 0.5643
Epoch 3/30
7352/7352 [==============================] - 117s 16ms/step - loss: 0.9794 - accuracy: 0.5681 - val_loss: 0.9824 - val_accuracy: 0.5168
Epoch 4/30
7352/7352 [==============================] - 118s 16ms/step - loss: 1.0273 - accuracy: 0.5491 - val_loss: 0.9189 - val_accuracy: 0.5775
Epoch 5/30
7352/7352 [==============================] - 119s 16ms/step - loss: 0.8047 - accuracy: 0.6411 - val_loss: 0.8748 - val_accuracy: 0.5969
Epoch 6/30
7352/7352 [==============================] - 119s 16ms/step - loss: 0.8253 - accuracy: 0.6299 - val_loss: 0.8473 - val_accuracy: 0.6057
Epoch 7/30
7352/7352 [==============================] - 134s 18ms/step

In [45]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 484       25         6        1                   0   
SITTING                  0      370        92        2                   1   
STANDING                 0       65       398        8                   0   
WALKING                  0        0         0      431                   9   
WALKING_DOWNSTAIRS       0        0         0       25                 388   
WALKING_UPSTAIRS         0        2         0       30                  12   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            21  
SITTING                           26  
STANDING                          61  
WALKING                           56  
WALKING_DOWNSTAIRS                 7  
WALKING_UPSTAIRS                 427  


In [46]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 8s 3ms/step


In [47]:
score

[0.8671318305163158, 0.84764164686203]

### Adding More layers

In [48]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 64

In [52]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.4))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 64)                18944     
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [54]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 181s 25ms/step - loss: 1.2111 - accuracy: 0.4767 - val_loss: 1.1095 - val_accuracy: 0.5497
Epoch 2/30
7352/7352 [==============================] - 191s 26ms/step - loss: 0.8816 - accuracy: 0.6336 - val_loss: 0.8614 - val_accuracy: 0.6400
Epoch 3/30
7352/7352 [==============================] - 184s 25ms/step - loss: 0.6760 - accuracy: 0.7191 - val_loss: 1.2741 - val_accuracy: 0.5185
Epoch 4/30
7352/7352 [==============================] - 180s 24ms/step - loss: 0.5214 - accuracy: 0.8161 - val_loss: 0.5649 - val_accuracy: 0.8273
Epoch 5/30
7352/7352 [==============================] - 181s 25ms/step - loss: 0.3538 - accuracy: 0.8876 - val_loss: 0.3780 - val_accuracy: 0.8799
Epoch 6/30
7352/7352 [==============================] - 176s 24ms/step - loss: 0.2669 - accuracy: 0.9116 - val_loss: 0.5281 - val_accuracy: 0.8585
Epoch 7/30
7352/7352 [==============================] - 172s 23ms/step

In [55]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  0      394        96        1                   0   
STANDING                 0       63       468        1                   0   
WALKING                  0        0         0      489                   0   
WALKING_DOWNSTAIRS       0        0         0        9                 410   
WALKING_UPSTAIRS         0        0         0       47                   4   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                            7  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 420  


In [56]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 12s 4ms/step


In [57]:
score

[0.31043342269294943, 0.9131320118904114]

In [24]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 64

In [125]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden,return_sequences=True,input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.55))
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_45 (LSTM)               (None, 128, 64)           18944     
_________________________________________________________________
dropout_39 (Dropout)         (None, 128, 64)           0         
_________________________________________________________________
lstm_46 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_40 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 6)                 390       
Total params: 52,358
Trainable params: 52,358
Non-trainable params: 0
_________________________________________________________________


In [126]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [68]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 512        0         0        0                   0   
SITTING                  3      417        67        1                   0   
STANDING                 0      111       420        0                   0   
WALKING                  0        0         1      467                  17   
WALKING_DOWNSTAIRS       0        0         0        2                 415   
WALKING_UPSTAIRS         0        1         2        4                  12   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            25  
SITTING                            3  
STANDING                           1  
WALKING                           11  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 452  


In [19]:
Y_train_dynamic=[]
X_train_dynamic=[]
Y_train_static=[]
X_train_static=[]
for i in range(0,len(Y_train)):
    if(Y_train[i][0]==1 or Y_train[i][1]==1 or Y_train[i][2]==1):
        Y_train_dynamic.append(Y_train[i])
        X_train_dynamic.append(X_train[i])
    else:
        Y_train_static.append(Y_train[i])
        X_train_static.append(X_train[i])

In [20]:
X_train_dynamic=np.array(X_train_dynamic)
Y_train_dynamic=np.array(Y_train_dynamic)
X_train_static=np.array(X_train_static)
Y_train_static=np.array(Y_train_static)


In [21]:
Y_train_dynamic=Y_train_dynamic[0:len(Y_train_dynamic),0:3]

In [22]:
Y_train_dynamic.shape

(3285, 3)

In [27]:
Y_test_dynamic=Y_test_dynamic[0:len(Y_test_dynamic),0:3]

In [24]:
Y_test_dynamic=[]
X_test_dynamic=[]
Y_test_static=[]
X_test_static=[]
for i in range(0,len(Y_test)):
    if(Y_test[i][0]==1 or Y_test[i][1]==1 or Y_test[i][2]==1):
        Y_test_dynamic.append(Y_test[i])
        X_test_dynamic.append(X_test[i])
    else:
        Y_test_static.append(Y_test[i])
        X_test_static.append(X_test[i])

In [26]:
X_test_dynamic=np.array(X_test_dynamic)
Y_test_dynamic=np.array(Y_test_dynamic)
X_test_static=np.array(X_test_static)
Y_test_static=np.array(Y_test_static)

In [28]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 24s 8ms/step


In [29]:
score

[0.3660856851979343, 0.9205971956253052]

In [257]:
# Initializing parameters
epochs = 30
batch_size = 32
n_hidden = 64

In [258]:
from keras.regularizers import l2

In [259]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(128,return_sequences=True,input_shape=(timesteps, input_dim),bias_regularizer=l2(0.001)))
# Adding a dropout layer
model.add(Dropout(0.2))

model.add(LSTM(64,input_shape=(timesteps, input_dim),bias_regularizer=l2(0.001)))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(Dense(50, activation='sigmoid'))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_78"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_53 (LSTM)               (None, 128, 128)          70656     
_________________________________________________________________
dropout_81 (Dropout)         (None, 128, 128)          0         
_________________________________________________________________
lstm_54 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dropout_82 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_121 (Dense)            (None, 50)                3250      
_________________________________________________________________
dense_122 (Dense)            (None, 6)                 306       
Total params: 123,620
Trainable params: 123,620
Non-trainable params: 0
_______________________________________________

In [260]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [261]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(7352, 128, 9)
(7352, 6)
(2947, 128, 9)
(2947, 6)


In [262]:
# Training the model
history=model.fit(X_train,
          Y_train,
          batch_size=25,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 373s 51ms/step - loss: 1.4446 - accuracy: 0.4539 - val_loss: 1.1004 - val_accuracy: 0.5338
Epoch 2/30
7352/7352 [==============================] - 383s 52ms/step - loss: 0.9190 - accuracy: 0.6235 - val_loss: 0.8570 - val_accuracy: 0.6932
Epoch 3/30
7352/7352 [==============================] - 346s 47ms/step - loss: 0.7800 - accuracy: 0.6782 - val_loss: 0.6684 - val_accuracy: 0.7472
Epoch 4/30
7352/7352 [==============================] - 337s 46ms/step - loss: 0.6132 - accuracy: 0.7606 - val_loss: 0.5975 - val_accuracy: 0.7645
Epoch 5/30
7352/7352 [==============================] - 362s 49ms/step - loss: 0.5592 - accuracy: 0.7724 - val_loss: 0.7208 - val_accuracy: 0.6715
Epoch 6/30
7352/7352 [==============================] - 351s 48ms/step - loss: 0.4514 - accuracy: 0.8458 - val_loss: 0.4499 - val_accuracy: 0.8751
Epoch 7/30
7352/7352 [==============================] - 391s 53ms/step

In [21]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense, Activation, Flatten
from keras.layers import LeakyReLU
model = Sequential()
model.add(Conv1D(filters=18,kernel_size=1, activation=LeakyReLU(alpha=0.3)
                 , input_shape=(timesteps,input_dim),kernel_initializer='glorot_uniform'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=36, kernel_size=3,kernel_initializer='glorot_uniform' ,activation=LeakyReLU(alpha=0.3)))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Conv1D(filters=36, kernel_size=3, kernel_initializer='glorot_uniform',activation=LeakyReLU(alpha=0.3)))
model.add(Dropout(0.5))
model.add(Conv1D(filters=144, kernel_size=3,kernel_initializer='glorot_uniform' ,activation=LeakyReLU(alpha=0.3),strides=2))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()


Instructions for updating:
Colocations handled automatically by placer.


/Users/bhawesh/anaconda3/lib/python3.7/site-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 128, 18)           180       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 18)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 62, 36)            1980      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 31, 36)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 29, 36)            3924      
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 36)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 14, 144)          

In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
history=model.fit(X_train,
          Y_train,
          batch_size=50,
          validation_data=(X_test, Y_test),
          epochs=15)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/15
7352/7352 [==============================] - 22s 3ms/step - loss: 1.2981 - accuracy: 0.5016 - val_loss: 0.9130 - val_accuracy: 0.6508
Epoch 2/15
7352/7352 [==============================] - 19s 3ms/step - loss: 0.6765 - accuracy: 0.7889 - val_loss: 0.5744 - val_accuracy: 0.8235
Epoch 3/15
7352/7352 [==============================] - 18s 2ms/step - loss: 0.4748 - accuracy: 0.8674 - val_loss: 0.5086 - val_accuracy: 0.8181
Epoch 4/15
7352/7352 [==============================] - 18s 2ms/step - loss: 0.3690 - accuracy: 0.9006 - val_loss: 0.4396 - val_accuracy: 0.8476
Epoch 5/15
7352/7352 [==============================] - 18s 2ms/step - loss: 0.2896 - accuracy: 0.9192 - val_loss: 0.3617 - val_accuracy: 0.8819
Epoch 6/15
7352/7352 [==============================] - 18s 2ms/step - loss: 0.2357 - accuracy: 0.9336 - val_loss: 0.3510 - val_accuracy: 0.8758
Epoch 7/15
7352/7352 [============

In [57]:
history=model.fit(X_train,
          Y_train,
          batch_size=16,
          validation_data=(X_test, Y_test),
          epochs=15)

Train on 7352 samples, validate on 2947 samples
Epoch 1/15
7352/7352 [==============================] - 26s 4ms/step - loss: 0.1001 - accuracy: 0.9596 - val_loss: 0.3262 - val_accuracy: 0.9077
Epoch 2/15
7352/7352 [==============================] - 28s 4ms/step - loss: 0.1092 - accuracy: 0.9551 - val_loss: 0.3122 - val_accuracy: 0.9182
Epoch 3/15
7352/7352 [==============================] - 27s 4ms/step - loss: 0.1011 - accuracy: 0.9554 - val_loss: 0.2787 - val_accuracy: 0.9189
Epoch 4/15
7352/7352 [==============================] - 26s 4ms/step - loss: 0.1040 - accuracy: 0.9553 - val_loss: 0.3150 - val_accuracy: 0.9148
Epoch 5/15
7352/7352 [==============================] - 25s 3ms/step - loss: 0.0999 - accuracy: 0.9567 - val_loss: 0.2987 - val_accuracy: 0.9108
Epoch 6/15
7352/7352 [==============================] - 25s 3ms/step - loss: 0.0891 - accuracy: 0.9608 - val_loss: 0.3206 - val_accuracy: 0.9121
Epoch 7/15
7352/7352 [==============================] - 28s 4ms/step - loss: 0.105

In [249]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  6      424        60        0                   0   
STANDING                 0      117       415        0                   0   
WALKING                  0        0         0      485                   7   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         0        1                  26   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                            4  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 444  


In [250]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 3s 1ms/step


In [251]:
score

[0.3760031298141544, 0.9246691465377808]

### Divide and Conquor

In [ ]:
#https://scholarcommons.usf.edu/cgi/viewcontent.cgi?article=8755&context=etd

### Model for prediction of Static or dynamic

In [22]:
model_d = Sequential()
model_d.add(Conv1D(filters=64, kernel_size=5, activation='relu',kernel_initializer='lecun_uniform',input_shape=(128,9)))
model_d.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='lecun_uniform'))
model_d.add(Dropout(0.45))
model_d.add(MaxPooling1D(pool_size=3))
model_d.add(Flatten())
model_d.add(Dense(32, activation='relu'))
model_d.add(Dense(2, activation='softmax'))
model_d.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 124, 64)           2944      
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 122, 32)           6176      
_________________________________________________________________
dropout_3 (Dropout)          (None, 122, 32)           0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 40, 32)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                40992     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                

In [23]:
## Classifying data as 2 class dynamic vs static 
##data preparation
def data_scaled_2class():
    # Data directory
    DATADIR = 'UCI_HAR_Dataset'
    
    # Raw data signals
    # Signals are from Accelerometer and Gyroscope
    # The signals are in x,y,z directions
    # Sensor signals are filtered to have only body acceleration
    # excluding the acceleration due to gravity
    # Triaxial acceleration from the accelerometer is total acceleration
    SIGNALS = [
        "body_acc_x",
        "body_acc_y",
        "body_acc_z",
        "body_gyro_x",
        "body_gyro_y",
        "body_gyro_z",
        "total_acc_x",
        "total_acc_y",
        "total_acc_z"
        ]
        
    # Utility function to read the data from csv file
    def _read_csv(filename):
        return pd.read_csv(filename, delim_whitespace=True, header=None)

    # Utility function to load the load
    def load_signals(subset):
        signals_data = []

        for signal in SIGNALS:
            filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
            signals_data.append( _read_csv(filename).as_matrix()) 

        # Transpose is used to change the dimensionality of the output,
        # aggregating the signals by combination of sample/timestep.
        # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
        return np.transpose(signals_data, (1, 2, 0))
    
    def load_y(subset):
        """
        The objective that we are trying to predict is a integer, from 1 to 6,
        that represents a human activity. We return a binary representation of 
        every sample objective as a 6 bits vector using One Hot Encoding
        (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
        """
        filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
        y = _read_csv(filename)[0]
        for i in range(0,len(y)):
            if(y[i]<=3):
                y[i] = 0
            else:
                y[i]=1
        return pd.get_dummies(y).as_matrix()
    
    X_train_2c, X_val_2c = load_signals('train'), load_signals('test')
    Y_train_2c, Y_val_2c = load_y('train'), load_y('test')
    
    return X_train_2c, Y_train_2c, X_val_2c,  Y_val_2c

In [24]:
X_train_2c, Y_train_2c, X_val_2c,  Y_val_2c = data_scaled_2class()

/Users/bhawesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/bhawesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [25]:
print(X_train_2c.shape)
print(Y_val_2c.shape)

(7352, 128, 9)
(2947, 2)


In [202]:
from keras.callbacks import *
filepath="epochs:{epoch:03d}-val_acc:{val_accuracy:.3f}.hdf7"
checkpoint_2 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, mode='max')

In [203]:
callbacks_list=[checkpoint_2]

In [204]:
model_d.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [205]:
model_d.fit(X_train_2c,Y_train_2c, epochs=20, batch_size=16,validation_data=(X_val_2c, Y_val_2c),
            verbose=1,callbacks=callbacks_list)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 23s 3ms/step - loss: 0.0246 - accuracy: 0.9895 - val_loss: 0.0512 - val_accuracy: 0.9885

Epoch 00001: saving model to epochs:001-val_acc:0.988.hdf7
Epoch 2/20
7352/7352 [==============================] - 23s 3ms/step - loss: 9.4761e-04 - accuracy: 0.9996 - val_loss: 0.0492 - val_accuracy: 0.9878

Epoch 00002: saving model to epochs:002-val_acc:0.988.hdf7
Epoch 3/20
7352/7352 [==============================] - 22s 3ms/step - loss: 0.0064 - accuracy: 0.9985 - val_loss: 0.0075 - val_accuracy: 0.9976

Epoch 00003: saving model to epochs:003-val_acc:0.998.hdf7
Epoch 4/20
7352/7352 [==============================] - 22s 3ms/step - loss: 8.6226e-04 - accuracy: 0.9997 - val_loss: 0.0025 - val_accuracy: 0.9990

Epoch 00004: saving model to epochs:004-val_acc:0.999.hdf7
Epoch 5/20
7352/7352 [==============================] - 24s 3ms/step - loss: 8.3795e-04 - accuracy: 0.9997 - val_loss: 0.002

In [22]:
best_model_division = load_model('epochs:009-val_acc:0.999.hdf7')

Instructions for updating:
Use tf.cast instead.


In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [10]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

/Users/bhawesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/bhawesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/bhawesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/bhawesh/anaconda3/lib/python3.7/site-packages/tensorflow/python/frame

In [11]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [12]:
# Import Keras
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from tensorflow.keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [21]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.models import Model, load_model

In [14]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 64

In [15]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [16]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/Users/bhawesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
/Users/bhawesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [17]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [18]:
X_train[0].shape

(128, 9)

In [34]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense, Activation, Flatten
from keras.layers import LeakyReLU
model1 = Sequential()
model1.add(Conv1D(filters=18,kernel_size=1, activation=LeakyReLU(alpha=0.3)
                 , input_shape=(timesteps,input_dim),kernel_initializer='he_uniform'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(Conv1D(filters=36, kernel_size=3,kernel_initializer='he_uniform' ,activation=LeakyReLU(alpha=0.3)))
model1.add(Dropout(0.5))
model1.add(Conv1D(filters=144, kernel_size=3,kernel_initializer='he_uniform' ,activation=LeakyReLU(alpha=0.3),strides=2))
model1.add(MaxPooling1D(pool_size=2,strides=2))
model1.add(Dropout(0.5))

model1.add(Flatten())
model1.add(Dense(32, activation='relu'))
model1.add(Dense(3, activation='sigmoid'))
model1.summary()

/Users/bhawesh/anaconda3/lib/python3.7/site-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 128, 18)           180       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 18)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 62, 36)            1980      
_________________________________________________________________
dropout_3 (Dropout)          (None, 62, 36)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 30, 144)           15696     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 15, 144)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 144)          

In [44]:
model1 = Sequential()
model1.add(Conv1D(filters=64, kernel_size=5, activation='relu',kernel_initializer='lecun_uniform',input_shape=(128,9)))
model1.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='lecun_uniform'))
model1.add(Dropout(0.45))
model1.add(MaxPooling1D(pool_size=3))
model1.add(Flatten())
model1.add(Dense(32, activation='relu'))
model1.add(Dense(3, activation='softmax'))
model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 124, 64)           2944      
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 122, 32)           6176      
_________________________________________________________________
dropout_6 (Dropout)          (None, 122, 32)           0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 40, 32)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                40992     
_________________________________________________________________
dense_7 (Dense)              (None, 3)                

In [23]:
Y_pred_train=best_model_division.predict(X_train)
Y_pred_test=best_model_division.predict(X_test)

In [25]:
Y_pred_train =  np.argmax(Y_pred_train, axis=1)
Y_pred_test =  np.argmax(Y_pred_test, axis=1)

In [26]:
Y_static_tr=Y_train[Y_pred_train==1]
Y_dynamic_tr=Y_train[Y_pred_train==0]
Y_static_test=Y_test[Y_pred_test==1]
Y_dynamic_test=Y_test[Y_pred_test==0]

In [39]:
Y_dynamic_tr=Y_dynamic_tr[0:len(Y_dynamic_tr),0:3]
Y_dynamic_test=Y_dynamic_test[0:len(Y_dynamic_test),0:3]
Y_static_tr=Y_static_tr[0:len(Y_static_tr),3:6]
Y_static_test=Y_static_test[0:len(Y_static_test),3:6]

In [28]:
X_static_tr=X_train[Y_pred_train==1]
X_dynamic_tr=X_train[Y_pred_train==0]
X_static_test=X_test[Y_pred_test==1]
X_dynamic_test=X_test[Y_pred_test==0]

In [29]:
from keras.callbacks import *
filepath="epochs:{epoch:03d}-val_acc:{val_accuracy:.3f}.hdf5"
checkpoint_2 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, mode='max')

In [30]:
callbacks_list = [checkpoint_2]

In [45]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
history=model1.fit(X_dynamic_tr,
          Y_dynamic_tr,
          batch_size=20,
          validation_data=(X_dynamic_test, Y_dynamic_test),
          epochs=20,callbacks=callbacks_list)

Train on 3285 samples, validate on 1391 samples
Epoch 1/20
3285/3285 [==============================] - 12s 4ms/step - loss: 0.4729 - accuracy: 0.7988 - val_loss: 0.3071 - val_accuracy: 0.9037

Epoch 00001: saving model to epochs:001-val_acc:0.904.hdf5
Epoch 2/20
3285/3285 [==============================] - 10s 3ms/step - loss: 0.0362 - accuracy: 0.9903 - val_loss: 0.2010 - val_accuracy: 0.9303

Epoch 00002: saving model to epochs:002-val_acc:0.930.hdf5
Epoch 3/20
3285/3285 [==============================] - 10s 3ms/step - loss: 0.0157 - accuracy: 0.9954 - val_loss: 0.3736 - val_accuracy: 0.8778

Epoch 00003: saving model to epochs:003-val_acc:0.878.hdf5
Epoch 4/20
3285/3285 [==============================] - 10s 3ms/step - loss: 0.0099 - accuracy: 0.9976 - val_loss: 0.1543 - val_accuracy: 0.9490

Epoch 00004: saving model to epochs:004-val_acc:0.949.hdf5
Epoch 5/20
3285/3285 [==============================] - 10s 3ms/step - loss: 9.9896e-04 - accuracy: 1.0000 - val_loss: 0.1406 - val_

In [47]:
best_model_d = load_model('epochs:020-val_acc:0.965.hdf5')

In [48]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [51]:
print(confusion_matrix(Y_dynamic_test, best_model_d.predict(X_dynamic_test)))

Pred                WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                                             
WALKING                 478                  18                 4
WALKING_DOWNSTAIRS        2                 418                 0
WALKING_UPSTAIRS          1                  24               446


In [55]:
score = best_model_d.evaluate(X_dynamic_test, Y_dynamic_test)

1391/1391 [==============================] - 2s 1ms/step


In [56]:
score

[0.1413168757945732, 0.9647735357284546]

### Static

In [90]:
Y_train_static=Y_train_static[0:len(Y_train_static),3:6]
Y_test_static=Y_test_static[0:len(Y_test_static),3:6]

In [19]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense, Activation, Flatten
from keras.layers import LeakyReLU
model = Sequential()
model.add(Conv1D(filters=18,kernel_size=1, activation=LeakyReLU(alpha=0.3)
                 , input_shape=(timesteps,input_dim),kernel_initializer='glorot_uniform'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=36, kernel_size=3,kernel_initializer='glorot_uniform' ,activation=LeakyReLU(alpha=0.3)))
model.add(Dropout(0.5))
model.add(Conv1D(filters=144, kernel_size=3,kernel_initializer='glorot_uniform' ,activation=LeakyReLU(alpha=0.3),strides=2))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


/Users/bhawesh/anaconda3/lib/python3.7/site-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 128, 18)           180       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 18)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 62, 36)            1980      
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 36)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 30, 144)           15696     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 15, 144)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 144)          

In [167]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [67]:
history=model.fit(X_static_tr,
          Y_train_static,
          batch_size=300,
          validation_data=(X_static_test, Y_static_test),
          epochs=15)

Train on 4067 samples, validate on 1560 samples
Epoch 1/15
4067/4067 [==============================] - 16s 4ms/step - loss: 0.3267 - accuracy: 0.8439 - val_loss: 0.3313 - val_accuracy: 0.8577
Epoch 2/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.2088 - accuracy: 0.9085 - val_loss: 0.4369 - val_accuracy: 0.8327
Epoch 3/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.2031 - accuracy: 0.9112 - val_loss: 0.3389 - val_accuracy: 0.8885
Epoch 4/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.1989 - accuracy: 0.9090 - val_loss: 0.3526 - val_accuracy: 0.8679
Epoch 5/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.1902 - accuracy: 0.9157 - val_loss: 0.3836 - val_accuracy: 0.8744
Epoch 6/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.1905 - accuracy: 0.9216 - val_loss: 0.3504 - val_accuracy: 0.8891
Epoch 7/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.186

In [57]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=5, activation='relu',kernel_initializer='glorot_uniform',input_shape=(128,9)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dropout(0.45))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 124, 64)           2944      
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 122, 32)           6176      
_________________________________________________________________
dropout_7 (Dropout)          (None, 122, 32)           0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 40, 32)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                40992     
_________________________________________________________________
dense_9 (Dense)              (None, 3)                

In [60]:
from keras.callbacks import *
filepath="epochs:{epoch:03d}-val_acc:{val_accuracy:.3f}.hdf6"
checkpoint_2 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, mode='max')

In [61]:
callbacks_list = [checkpoint_2]

In [62]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [64]:
history=model.fit(X_static_tr,
          Y_static_tr,
          batch_size=50,
          validation_data=(X_static_test, Y_static_test),
          epochs=15,callbacks=callbacks_list)

Train on 4067 samples, validate on 1556 samples
Epoch 1/15
4067/4067 [==============================] - 12s 3ms/step - loss: 0.2928 - accuracy: 0.8866 - val_loss: 0.3406 - val_accuracy: 0.8599

Epoch 00001: saving model to epochs:001-val_acc:0.860.hdf6
Epoch 2/15
4067/4067 [==============================] - 10s 3ms/step - loss: 0.2238 - accuracy: 0.9036 - val_loss: 0.3242 - val_accuracy: 0.8683

Epoch 00002: saving model to epochs:002-val_acc:0.868.hdf6
Epoch 3/15
4067/4067 [==============================] - 10s 2ms/step - loss: 0.1978 - accuracy: 0.9157 - val_loss: 0.3095 - val_accuracy: 0.8817

Epoch 00003: saving model to epochs:003-val_acc:0.882.hdf6
Epoch 4/15
4067/4067 [==============================] - 10s 2ms/step - loss: 0.1867 - accuracy: 0.9186 - val_loss: 0.3133 - val_accuracy: 0.8760

Epoch 00004: saving model to epochs:004-val_acc:0.876.hdf6
Epoch 5/15
4067/4067 [==============================] - 11s 3ms/step - loss: 0.1792 - accuracy: 0.9213 - val_loss: 0.3005 - val_accu

In [65]:
history=model.fit(X_static_tr,
          Y_static_tr,
          batch_size=50,
          validation_data=(X_static_test, Y_static_test),
          epochs=15,callbacks=callbacks_list)

Train on 4067 samples, validate on 1556 samples
Epoch 1/15
4067/4067 [==============================] - 10s 2ms/step - loss: 0.1106 - accuracy: 0.9503 - val_loss: 0.3441 - val_accuracy: 0.8959

Epoch 00001: saving model to epochs:001-val_acc:0.896.hdf6
Epoch 2/15
4067/4067 [==============================] - 9s 2ms/step - loss: 0.1069 - accuracy: 0.9489 - val_loss: 0.3390 - val_accuracy: 0.9042

Epoch 00002: saving model to epochs:002-val_acc:0.904.hdf6
Epoch 3/15
4067/4067 [==============================] - 10s 3ms/step - loss: 0.0985 - accuracy: 0.9550 - val_loss: 0.4258 - val_accuracy: 0.8888

Epoch 00003: saving model to epochs:003-val_acc:0.889.hdf6
Epoch 4/15
4067/4067 [==============================] - 10s 3ms/step - loss: 0.0901 - accuracy: 0.9607 - val_loss: 0.4983 - val_accuracy: 0.8830

Epoch 00004: saving model to epochs:004-val_acc:0.883.hdf6
Epoch 5/15
4067/4067 [==============================] - 10s 2ms/step - loss: 0.0913 - accuracy: 0.9594 - val_loss: 0.5533 - val_accur

In [66]:
history=model.fit(X_static_tr,
          Y_static_tr,
          batch_size=20,
          validation_data=(X_static_test, Y_static_test),
          epochs=15,callbacks=callbacks_list)

Train on 4067 samples, validate on 1556 samples
Epoch 1/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.1032 - accuracy: 0.9584 - val_loss: 0.3711 - val_accuracy: 0.8843

Epoch 00001: saving model to epochs:001-val_acc:0.884.hdf6
Epoch 2/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.1028 - accuracy: 0.9584 - val_loss: 0.3037 - val_accuracy: 0.9132

Epoch 00002: saving model to epochs:002-val_acc:0.913.hdf6
Epoch 3/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.1246 - accuracy: 0.9530 - val_loss: 0.2716 - val_accuracy: 0.9017

Epoch 00003: saving model to epochs:003-val_acc:0.902.hdf6
Epoch 4/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.0846 - accuracy: 0.9614 - val_loss: 0.2659 - val_accuracy: 0.9068

Epoch 00004: saving model to epochs:004-val_acc:0.907.hdf6
Epoch 5/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.0705 - accuracy: 0.9678 - val_loss: 0.3229 - val_accu

In [68]:
history=model.fit(X_static_tr,
          Y_static_tr,
          batch_size=20,
          validation_data=(X_static_test, Y_static_test),
          epochs=15,callbacks=callbacks_list)

Train on 4067 samples, validate on 1556 samples
Epoch 1/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.0540 - accuracy: 0.9766 - val_loss: 0.3624 - val_accuracy: 0.9235

Epoch 00001: saving model to epochs:001-val_acc:0.924.hdf6
Epoch 2/15
4067/4067 [==============================] - 10s 3ms/step - loss: 0.0595 - accuracy: 0.9739 - val_loss: 0.3420 - val_accuracy: 0.9171

Epoch 00002: saving model to epochs:002-val_acc:0.917.hdf6
Epoch 3/15
4067/4067 [==============================] - 14s 3ms/step - loss: 0.0528 - accuracy: 0.9779 - val_loss: 0.4323 - val_accuracy: 0.9242

Epoch 00003: saving model to epochs:003-val_acc:0.924.hdf6
Epoch 4/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.0537 - accuracy: 0.9779 - val_loss: 0.4035 - val_accuracy: 0.9120

Epoch 00004: saving model to epochs:004-val_acc:0.912.hdf6
Epoch 5/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.0551 - accuracy: 0.9754 - val_loss: 0.4302 - val_accu

In [70]:
history=model.fit(X_static_tr,
          Y_static_tr,
          batch_size=16,
          validation_data=(X_static_test, Y_static_test),
          epochs=15,callbacks=callbacks_list)

Train on 4067 samples, validate on 1556 samples
Epoch 1/15
4067/4067 [==============================] - 14s 3ms/step - loss: 0.0449 - accuracy: 0.9838 - val_loss: 0.6696 - val_accuracy: 0.9055

Epoch 00001: saving model to epochs:001-val_acc:0.906.hdf6
Epoch 2/15
4067/4067 [==============================] - 14s 3ms/step - loss: 0.0650 - accuracy: 0.9828 - val_loss: 0.6728 - val_accuracy: 0.9030

Epoch 00002: saving model to epochs:002-val_acc:0.903.hdf6
Epoch 3/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.1393 - accuracy: 0.9717 - val_loss: 0.7011 - val_accuracy: 0.8972

Epoch 00003: saving model to epochs:003-val_acc:0.897.hdf6
Epoch 4/15
4067/4067 [==============================] - 12s 3ms/step - loss: 0.1116 - accuracy: 0.9796 - val_loss: 0.7002 - val_accuracy: 0.9100

Epoch 00004: saving model to epochs:004-val_acc:0.910.hdf6
Epoch 5/15
4067/4067 [==============================] - 12s 3ms/step - loss: 0.1032 - accuracy: 0.9830 - val_loss: 0.6342 - val_accu

In [73]:
history=model.fit(X_static_tr,
          Y_static_tr,
          batch_size=20,
          validation_data=(X_static_test, Y_static_test),
          epochs=15,callbacks=callbacks_list)

Train on 4067 samples, validate on 1556 samples
Epoch 1/15
4067/4067 [==============================] - 12s 3ms/step - loss: 0.0773 - accuracy: 0.9921 - val_loss: 0.7688 - val_accuracy: 0.9177

Epoch 00001: saving model to epochs:001-val_acc:0.918.hdf6
Epoch 2/15
4067/4067 [==============================] - 11s 3ms/step - loss: 0.0753 - accuracy: 0.9934 - val_loss: 0.7692 - val_accuracy: 0.9139

Epoch 00002: saving model to epochs:002-val_acc:0.914.hdf6
Epoch 3/15
4067/4067 [==============================] - 11s 3ms/step - loss: 0.0744 - accuracy: 0.9931 - val_loss: 0.7468 - val_accuracy: 0.9107

Epoch 00003: saving model to epochs:003-val_acc:0.911.hdf6
Epoch 4/15
4067/4067 [==============================] - 13s 3ms/step - loss: 0.0796 - accuracy: 0.9904 - val_loss: 0.7810 - val_accuracy: 0.9126

Epoch 00004: saving model to epochs:004-val_acc:0.913.hdf6
Epoch 5/15
4067/4067 [==============================] - 14s 3ms/step - loss: 0.0826 - accuracy: 0.9914 - val_loss: 0.7787 - val_accu

In [74]:
best_model_s = load_model('epochs:011-val_acc:0.930.hdf6')

In [75]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'SITTING',
    1: 'STANDING',
    2: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [76]:
print(confusion_matrix(Y_static_test, model.predict(X_static_test)))

Pred      LAYING  SITTING  STANDING
True                               
LAYING       512        0        25
SITTING        0      389       102
STANDING       0       27       501


In [78]:
score = best_model_s.evaluate(X_static_test, Y_static_test)

1556/1556 [==============================] - 2s 1ms/step


In [79]:
score

[0.39326874985088, 0.9299485683441162]

### Final Pipelining

In [80]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return y
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [81]:
X_train, X_test, Y_train, Y_test = load_data()

/Users/bhawesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [82]:
len(Y_train)

7352

In [83]:
Y_pred_train=best_model_division.predict(X_train)
Y_pred_test=best_model_division.predict(X_test)

In [84]:
Y_pred_train =  np.argmax(Y_pred_train, axis=1)
Y_pred_test =  np.argmax(Y_pred_test, axis=1)

In [85]:
X_static_tr=X_train[Y_pred_train==1]
X_dynamic_tr=X_train[Y_pred_train==0]
X_static_test=X_test[Y_pred_test==1]
X_dynamic_test=X_test[Y_pred_test==0]

In [86]:
Y_pred_static_tr=best_model_s.predict(X_static_tr)
Y_pred_static_test=best_model_s.predict(X_static_test)

In [88]:
Y_pred_dynamic_tr=best_model_d.predict(X_dynamic_tr)
Y_pred_dynamic_test=best_model_d.predict(X_dynamic_test)

In [89]:
Y_pred_dynamic_tr=np.argmax(Y_pred_dynamic_tr, axis=1)
Y_pred_dynamic_test=np.argmax(Y_pred_dynamic_test, axis=1)
Y_pred_static_tr=np.argmax(Y_pred_static_tr, axis=1)
Y_pred_static_test=np.argmax(Y_pred_static_test, axis=1)

In [90]:
Y_pred_static_tr=Y_pred_static_tr+4
Y_pred_static_test=Y_pred_static_test+4

In [91]:
Y_pred_dynamic_tr=Y_pred_dynamic_tr+1
Y_pred_dynamic_test=Y_pred_dynamic_test+1

In [92]:
k,j = 0,0 
final_pred_tr = []
for i in Y_pred_train:
    if i == 1:
        final_pred_tr.append(Y_pred_static_tr[k])
        k = k + 1
    else:
        final_pred_tr.append(Y_pred_dynamic_tr[j])
        j = j + 1 

In [93]:
k,j = 0,0 
final_pred_test = []
for i in Y_pred_test:
    if i == 1:
        final_pred_test.append(Y_pred_static_test[k])
        k = k + 1
    else:
        final_pred_test.append(Y_pred_dynamic_test[j])
        j = j + 1 

In [94]:
##accuracy of train and test
from sklearn.metrics import accuracy_score
print('Accuracy of train data',accuracy_score(Y_train,final_pred_tr))
print('Accuracy of validation data',accuracy_score(Y_test,final_pred_test))

Accuracy of train data 0.9944232861806311
Accuracy of validation data 0.9463861554122837
